# Upwind scheme for Wave System on square meshes

## The Wave System on the square

We consider the following Wave system with wall boundary conditions

$$
\left\{\begin{array}{l}
\partial_t p + c^2\nabla\cdot\vec q = 0\\
\partial_t \vec q + \quad\vec\nabla p = 0
\end{array}\right..
$$

The wave system can be written in matrix form 
$$
\partial_t\left(
\begin{array}{c}
 p\\ \vec q
\end{array}\right)
+
\left(
\begin{array}{cc}
 0              & c^2\nabla\cdot \\ 
 \vec\nabla & 0
\end{array}\right)
\left(
\begin{array}{c}
 p\\\vec q
\end{array}\right)
=
\left(
\begin{array}{c}
 0 \\ \vec 0
\end{array}\right)
$$
In $d$ space dimensions the wave system is an hyperbolic system of $d+1$ equations
$$
\partial_t U +\sum_{i=1}^d A_i\partial_{x_i} U=0,\quad U={}^t(p,\vec q)
$$
where the jacobian matrix is
$$
A(\vec n)=\sum_{i=1}^d n_i A_i =
\left(
\begin{array}{cc}
 0              & c^2 {}^t\vec n \\ 
 \vec n & 0
\end{array}\right),\quad \vec n\in\mathbb{R}^d.
$$
has $d+1$ eigenvalues $-c,0,\dots,0,c$.

The wave system also takes the conservative form
$$
\partial_t U + \nabla\cdot F(U)=0,
$$
where the flux matrix $F$ is defined by
$$
F(U)\vec n=A(\vec n)U,\quad \vec n\in\mathbb{R}.
$$

On the square domain $\Omega= [0,1]\times [0,1]$ we consider the initial data 
$$
\left\{\begin{array}{l}
p_0(x,y)=constant\\
q_{x0}(x,y)= \sin(\pi x) \cos(\pi y)\\
q_{y0}(x,y)=-\sin(\pi y) \cos(\pi x)
\end{array}\right..
$$  
The initial data $(p_0,q_x,q_y)$ is a stationary solution of the wave system.



## The Upwind scheme for the Wave System

The domain $\Omega$ is decomposed into cells $C_i$.

$|C_i|$ is the measure of the cell $C_i$.

$f_{ij}$ is the interface between two cells $C_i$ and $C_j$. 

$s_{ij}$ is the measure of the interface $f_{ij}$.

$d_{ij}$ is the distance between the centers of mass of the two cells $C_i$ and $C_j$.

The semi-discrete colocated finite volume equation is
$$
\partial_t U + \frac{1}{|C_i|} \sum s_{ij}F_{ij}=0,
$$
where
$U_i$ is the approximation of $U$ in the cell $C_i$,

$F_{ij}$ is a numerical approximation of the outward normal interfacial flux from cell $i$ to cell $j$ usually in the form
$$

$$
In the case of the upwind scheme the interfacial flux is
$$
F_{ij}=\frac{u_j-u_i}{d_{ij}}.
$$

## The script

```python
#Discrétisation du second membre et extraction du nb max de voisins d'une cellule
#================================================================================
my_RHSfield = cdmath.Field("RHS_field", cdmath.CELLS, my_mesh, 1)
maxNbNeighbours=0#This is to determine the number of non zero coefficients in the sparse finite element rigidity matrix

for i in range(nbCells): 
	Ci = my_mesh.getCell(i)
	x = Ci.x()
	y = Ci.y()

	my_RHSfield[i]=2*pi*pi*sin(pi*x)*sin(pi*y)#mettre la fonction definie au second membre de l edp
	# compute maximum number of neighbours
	maxNbNeighbours= max(1+Ci.getNumberOfFaces(),maxNbNeighbours)

# Construction de la matrice et du vecteur second membre du système linéaire
#===========================================================================
Rigidite=cdmath.SparseMatrixPetsc(nbCells,nbCells,maxNbNeighbours)# warning : third argument is max number of non zero coefficients per line of the matrix
RHS=cdmath.Vector(nbCells)
#Parcours des cellules du domaine
for i in range(nbCells):
	RHS[i]=my_RHSfield[i] #la valeur moyenne du second membre f dans la cellule i
	Ci=my_mesh.getCell(i)
	for j in range(Ci.getNumberOfFaces()):# parcours des faces voisinnes
		Fj=my_mesh.getFace(Ci.getFaceId(j))
		if not Fj.isBorder():
			k=Fj.getCellId(0)
			if k==i :
				k=Fj.getCellId(1)
			Ck=my_mesh.getCell(k)
			distance=Ci.getBarryCenter().distance(Ck.getBarryCenter())
			coeff=Fj.getMeasure()/Ci.getMeasure()/distance
			Rigidite.setValue(i,k,-coeff) # terme extradiagonal
		else:
			coeff=Fj.getMeasure()/Ci.getMeasure()/Ci.getBarryCenter().distance(Fj.getBarryCenter())
		Rigidite.addValue(i,i,coeff) # terme diagonal


# Résolution du système linéaire
#=================================
LS=cdmath.LinearSolver(Rigidite,RHS,500,1.E-6,"GMRES","ILU")
SolSyst=LS.solve()

# Automatic postprocessing :  save 2D picture and plot diagonal data
#===========================
diag_data=VTK_routines.Extract_field_data_over_line_to_numpyArray(my_ResultField,[0,1,0],[1,0,0], resolution)
plt.legend()
plt.xlabel('Position on diagonal line')
plt.ylabel('Value on diagonal line')
if len(sys.argv) >1 :
    plt.title('Plot over diagonal line for finite Volumes \n for Laplace operator on a 2D square '+my_mesh.getName())
    plt.plot(curv_abs, diag_data, label= str(nbCells)+ ' cells mesh')
    plt.savefig("FiniteVolumes2D_square_ResultField_"+str(nbCells)+ '_cells'+"_PlotOverDiagonalLine.png")

```

## Numerical results

### Cartesian meshes

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](squareWithSquares_1.png) | ![](squareWithSquares_2.png)  | ![](squareWithSquares_3.png) 


### Numerical results

result 1 | result 2 | result 3
       - | -      - | -
![](FiniteVolumes2D_square_ResultField16.png) | ![](FiniteVolumes2D_square_ResultField64.png)  | ![](FiniteVolumes2D_square_ResultField256.png) 


## Convergence

![](SquareWithSquares_2DPoissonFV_ConvergenceCurve.png)

![](SquareWithSquares_2DPoissonFV_PlotOverDiagonalLine.png)